# Data Masters: Case

## Bibliotecas

In [1]:
# --- Data Exploration and Viz --- #
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# --- Classification models --- #
from sklearn.ensemble import \
    GradientBoostingClassifier, \
    RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# --- Pipeline Building --- #
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# --- Model Evaluation --- #
from sklearn.metrics import confusion_matrix, roc_auc_score

# --- Tuning --- #
from sklearn.model_selection import GridSearchCV

# --- Preprocessing --- #
from sklearn.preprocessing import \
    StandardScaler, \
    OrdinalEncoder, \
    FunctionTransformer
 
from resources.customtransformers import \
    DropConstantColumns, \
    DropDuplicateColumns, \
    AddNoneCount, \
    AddNonZeroCount

# --- Cluster Analysis --- #
#from sklearn.decomposition import PCA
#from sklearn.cluster import KMeans

## Leitura dos dados

In [2]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

## Definição do pipeline inicial

In [51]:
prep_base = Pipeline(
    steps=[
        ("dcc", DropConstantColumns()),
        ("ddc", DropDuplicateColumns()),
        ("anzc_saldo", AddNonZeroCount(prefix="saldo")),
        ("anzc_imp", AddNonZeroCount(prefix="imp")),
        (
            "anc_delta",
            AddNoneCount(
                prefix="delta",
                fake_value=9999999999,
                drop_constant=True
            )
        ),
        ("anzc_delta", AddNonZeroCount(prefix="delta")),
        ("anzc_ind", AddNonZeroCount(prefix="ind")),
        (
            "col_specific",
            ColumnTransformer(
                [
                    (
                        "ord_encoders",
                        OrdinalEncoder(
                            handle_unknown="use_encoded_value",
                            unknown_value=-1,
                            encoded_missing_value=-1,
                            min_frequency=40
                        ),
                        ["var36","var21"]                    
                    ),
                    (
                        "log_transformer",
                        FunctionTransformer(lambda x: np.log(x+1)),
                        ["var38"]
                    )
                ],
                remainder="passthrough"
            )
        )
    ]
)

In [52]:
prep_base = prep_base.fit(
    train.drop(["ID", "TARGET"],axis=1),
    y=train["TARGET"]
)

Pipeline(steps=[('dcc', DropConstantColumns()), ('ddc', DropDuplicateColumns()),
                ('anzc_saldo', AddNonZeroCount(prefix='saldo')),
                ('anzc_imp', AddNonZeroCount(prefix='imp')),
                ('anc_delta',
                 AddNoneCount(drop_constant=True, fake_value=9999999999,
                              prefix='delta')),
                ('anzc_delta', AddNonZeroCount(prefix='delta')),
                ('anzc_ind', AddNonZeroCount(prefix='ind')),
                ('col_specific',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ord_encoders',
                                                  OrdinalEncoder(encoded_missing_value=-1,
                                                                 handle_unknown='use_encoded_value',
                                                                 min_frequency=40,
                                                                 unknown_value=-1),
                                                  ['var36', 'var21']),
                                                 ('log_transformer',
                                                  FunctionTransformer(func=<function <lambda> at 0x7fd660c83250>),
                                                  ['var38'])]))])

In [126]:
def predict_probability(X, model):
    return model.predict_proba(X)[:,1]

def make_result_df(y,y_pred):
    df = y.reset_index()
    df["prob"] = y_pred
    df["pred"] = y_pred
    return df

def predict_class(y, c):
    y["pred"] = y["prob"].apply(lambda x: 1 if x >= c else 0)
    return y

def evaluate(df, c):
    tn, fp, fn, tp = confusion_matrix(
        df["TARGET"],
        predict_class(df, c)["pred"],
    ).ravel()
    return (tn*0+fp*(-10)+fn*0+tp*(100-10))/len(df)

def select_threshold(X, y):
    y_pred = predict_probability(X, gscv)
    df = make_result_df(y,y_pred)

    thresh_scores = [[i/100, 0] for i in range(101)]

    for i in range(len(thresh_scores)):
        c = thresh_scores[i][0]
        thresh_scores[i][1] = evaluate(df,c)

    thresh_scores = pd.DataFrame(thresh_scores,columns=["Threshold", "Profit"])

    best_threshold = thresh_scores.iloc[
        thresh_scores.Profit.idxmax()
    ].Threshold

    return best_threshold